In [2]:
from docx import Document
from docx.shared import Mm, Pt, Cm
from docx.enum.text import WD_LINE_SPACING, WD_ALIGN_PARAGRAPH, WD_UNDERLINE
from docx.enum.table import WD_CELL_VERTICAL_ALIGNMENT, WD_TABLE_ALIGNMENT

import numpy as np
import json
import os
from random import randrange
from datetime import timedelta, datetime

from write import write_pdf_linux, write_jpg

from pdf2jpg import pdf2jpg
import pypdfium2 as pdfium

import pymorphy2
from faker import Faker

In [26]:
# f = Faker('ru_RU')
# [f.company() for _ in range(30)]
# a = 'Макаров Инк'
# any([stop_word in a.lower() for stop_word in ['инк','лтд']])

In [275]:
# fake.company()

'НПО «Лебедев»'

In [217]:
# morph.parse(header_sample[3])[0].inflect({'datv'}).word
# adresat.title()
morph = pymorphy2.MorphAnalyzer(lang='ru')
sample = resp[np.random.choice(range(len(resp)))]
print(sample)
most_likely = morph.parse(sample[1])[0]
for candidat in morph.parse(sample[1]):
    if 'Surn' in candidat.tag or 'nomn' in candidat.tag: # 'Surn' 'Name', 'Patr'
        most_likely = candidat
        if {'Surn', 'nomn'} in candidat.tag:
            break
most_likely
most_likely.inflect({'datv', 'sing'})

['{Системный} {администратор} {{Сергеев}} {{Андрей}}', 'Сергеев', 'Андрей', '', 'Системный администратор', 'Служба ИТ']


Parse(word='сергееву', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Surn sing,datv'), normal_form='сергеев', score=1.0, methods_stack=((DictionaryAnalyzer(), 'сергееву', 37, 2),))

In [3]:
# формат json {'words': [], 'bbox': [[], []]}

def generate_mail_data():
    def random_date(start, end):
        """
        This function will return a random datetime between two datetime 
        objects.
        """
        delta = end - start
        int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
        random_second = randrange(int_delta)
        return start + timedelta(seconds=random_second)
    
    fake = Faker('ru_RU')
    
    # logo
    logo = os.path.join('samples', 'logo', np.random.choice(os.listdir(os.path.join('samples', 'logo'))))
    date = random_date(datetime.strptime('01.01.2000', '%d.%m.%Y'), datetime.strptime('01.11.2023', '%d.%m.%Y')).strftime('%d.%m.%Y')
    mail_num = fake.postcode()
    
    # header
    with open(os.path.join('samples', 'responsible.json'), 'r', encoding='utf-8') as f:
        resp = json.load(f)
        
    morph = pymorphy2.MorphAnalyzer(lang='ru')
    header_sample = resp[np.random.choice(np.arange(len(resp)))]
    
    title = ''
    if np.random.random() > 0.5:
        title_tokens = header_sample[4].split(' ')
        if 'ADJF' in morph.parse(title_tokens[0])[0].tag:
            title_tokens[0] = morph.parse(title_tokens[0])[0].inflect({'datv'}).word
            title_tokens[1] = morph.parse(title_tokens[1])[0].inflect({'datv'}).word
        else:
            title_tokens[0] = morph.parse(title_tokens[0])[0].inflect({'datv'}).word
        title = ' '.join(title_tokens)

    adresat_tokens = header_sample[1:4]
    adresat_part = ['Surn', 'Name', 'Patr']
    ref_name = header_sample[1:4]
    for i in range(len(adresat_tokens)):
        if len(adresat_tokens[i]) > 0:
            most_likely = morph.parse(adresat_tokens[i])[0]
            for candidat in morph.parse(adresat_tokens[i]):
                if adresat_part[i] in candidat.tag or 'nomn' in candidat.tag: # 'Surn' 'Name', 'Patr'
                    most_likely = candidat
                    if {adresat_part[i], 'nomn'} in candidat.tag:
                        break
            word_datv = most_likely.inflect({'datv', 'sing'})
    
            if word_datv:
                adresat_tokens[i] = word_datv.word
                ref_name[i] = most_likely.normal_form
    
    adresat = ' '.join(adresat_tokens)
    
    org = fake.company()
    while any([stop_word in org.lower() for stop_word in ['инк','лтд']]):
        org = fake.company()
    
    header = title.capitalize() + ' ' + org + ' ' + adresat.title() 
    
    # reference
    has_ref = True if np.random.random() > 0.5 else False
    reference_text = ''
    if has_ref:
        reference_text = 'Уважаемый ' + ' '.join(ref_name).title() + '!'
        
    # main_text
    main_text_samples = ["""Мы обращаемся к вам от имени Название вашей организации с важным сообщением. Мы хотели бы поделиться информацией о нашей компании и ее деятельности.
Ваша организация является ведущей в своей отрасли и специализируется на указать область деятельности. Мы гордимся нашими достижениями и стремимся к постоянному развитию и инновациям.
Мы предлагаем широкий спектр продуктов и услуг, которые удовлетворяют потребности наших клиентов. Наша команда состоит из высококвалифицированных специалистов, которые работают вместе, чтобы обеспечить высокое качество и превосходное обслуживание.
Спасибо за ваше внимание и поддержку. Мы надеемся на возможность сотрудничества с вами.""", 
                """Сообщаем Вам о том, что 15.07.2023 года в 10:00 состоится онлайн-конференция «Современные технологии и методы проектирования в машиностроении».
Программа конференции:
1. Введение в тему.
2. Инновационные технологии проектирования машин.
3. Применение современных CAD/CAM систем при проектировании механических конструкций.
Для участия в конференции необходимо направить заявку на электронную почту адрес до 12.07.2023 года."""]
    main_text = np.random.choice(main_text_samples)
    
    # creator
    footer_sample = resp[np.random.choice(np.arange(len(resp)))]
    creator_title = footer_sample[4]
    creator_name = footer_sample[1] + ' '
    for tok in footer_sample[2:4]:
        if bool(tok):
            creator_name += tok[0] + '. '
    
    creator_name = creator_name.strip()
    
    # signature
    signature = os.path.join('samples', 'signature', np.random.choice(os.listdir(os.path.join('samples', 'signature'))))
    
    # executor
    
    contact_type = np.random.choice(['mail', 'phone'])
    executor_name = fake.name()
    contact = fake.ascii_free_email() if contact_type == 'mail' else fake.phone_number()

    return logo, date, mail_num, header, has_ref, reference_text, main_text, creator_title, creator_name, signature, executor_name, contact


In [7]:
out_dir = 'letters_out'

def generate_doc(filename): # без разрешения файла
    logo, date, mail_num, header, has_ref, reference_text, main_text, creator_title, creator_name, signature, executor_name, contact = generate_mail_data()
    document = Document()
    
    text_tokens = [header, date, mail_num, reference_text, main_text, creator_title, creator_name, executor_name, contact]
    with open(os.path.join(out_dir, 'txt', f'{filename}.txt'), 'w') as f:
        for tok in text_tokens:
            if tok == text_tokens[-1]:
                f.write(tok)
            else:
                for word in tok.split(' '):
                    if len(word) > 0:
                        f.write(word + '\n')
                    
    # General doc settings
    style = document.styles['Normal']
    font = style.font
    p_format = style.paragraph_format
    font.name = "Times New Roman" # consts.font_name
    font.size = Pt(14) # consts.font_size
    p_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE # consts.line_spacing
    
    for section in document.sections:
        section.top_margin =  Mm(20) # consts.top_margin
        section.bottom_margin = Mm(20) # consts.bottom_margin
        section.left_margin = Mm(30) # consts.left_margin
        section.right_margin = Mm(15) # consts.right_margin
        
    # Header
    table = document.add_table(rows=1, cols=2)
    for cell in table.rows[0].cells:
        tc = cell._element.tcPr
        tc.left = None
        tc.top = None
        tc.right = None
        tc.bottom = None
        cell.vertical_alignment = WD_CELL_VERTICAL_ALIGNMENT.TOP
        cell.paragraphs[0].alignment = WD_TABLE_ALIGNMENT.LEFT
    
    table.cell(0, 0).paragraphs[0].add_run().add_picture(logo, Mm(40), Mm(40))
    run = table.cell(0, 0).add_paragraph().add_run(f'{date} № {mail_num} ')
    run.font.underline = True
    run.font.size = Pt(10)
    run = table.cell(0, 0).add_paragraph().add_run(f'на №        от        .')
    run.font.underline = True
    run.font.size = Pt(10)
    
    header_cell = table.cell(0, 1)
    header_cell.paragraphs[0].add_run(header)
    header_cell.paragraphs[0].alignment = WD_TABLE_ALIGNMENT.RIGHT
    
    for c0, c1 in zip(table.columns[0].cells, table.columns[1].cells):
        c0.width = Cm(10.5)
        c1.width = Cm(7.0)
    
    # Reference
    if has_ref:
        ref = document.add_paragraph()
        ref.alignment = 1
        ref.add_run(reference_text)
    
    # Main part
    main = document.add_paragraph()
    main_fmt = main.paragraph_format
    main_fmt.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    main_fmt.first_line_indent = Cm(1.25)
    main.add_run(main_text)
    
    # Footer
    foot_table = document.add_table(rows=2, cols=3)
    
    for cell in foot_table.rows[0].cells:
        tc = cell._element.tcPr
        tc.left = None
        tc.top = None
        tc.right = None
        tc.bottom = None
        cell.vertical_alignment = WD_CELL_VERTICAL_ALIGNMENT.TOP
        cell.paragraphs[0].alignment = WD_TABLE_ALIGNMENT.LEFT
    
    for c0, c1, c2 in zip(foot_table.columns[0].cells, foot_table.columns[1].cells, foot_table.columns[2].cells):
        c0.width = Cm(10.0)
        c1.width = Cm(3.0)
        c2.width = Cm(4.0)
    
    foot_table.cell(0, 0).paragraphs[0].add_run(creator_title)
    foot_table.cell(0, 0).paragraphs[0].alignment = WD_TABLE_ALIGNMENT.LEFT
    
    foot_table.cell(0, 1).paragraphs[0].add_run().add_picture(signature, Mm(30), Mm(20))
    foot_table.cell(0, 1).paragraphs[0].alignment = WD_TABLE_ALIGNMENT.RIGHT
    
    foot_table.cell(0, 2).paragraphs[0].add_run(creator_name)
    foot_table.cell(0, 2).paragraphs[0].alignment = WD_TABLE_ALIGNMENT.RIGHT
    
    
    run = foot_table.cell(1, 0).paragraphs[0].add_run(executor_name + '\n' + contact)
    foot_table.cell(1, 0).paragraphs[0].alignment = WD_TABLE_ALIGNMENT.LEFT
    run.font.size = Pt(10)
    
      
    document.save(os.path.join(out_dir, 'doc', f'{filename}.docx'))

# write_pdf_linux('letter.docx', '.', 0)
# write_jpg('.', 0)

In [5]:
# def generate_pdf(docx_path, filename):
#     wdFormatPDF = 17
#     docx_path = os.path.abspath(docx_path)
#     out_file = os.path.abspath(os.path.join(out_dir, 'pdf', f'{filename}.pdf'))
    
#     word = comtypes.client.CreateObject('Word.Application')
#     doc = word.Documents.Open(docx_path)
#     doc.SaveAs(out_file, FileFormat=wdFormatPDF)
#     doc.Close()
#     word.Quit()
def generate_pdf(docx_path, filename):
    convert(docx_path, os.path.join(out_dir, 'pdf', f'{filename}.pdf'))
def generate_jpg(pdf_path, filename):
    pdf = pdfium.PdfDocument(pdf_path)

    n_pages = len(pdf)
    
    for page_num in range(n_pages):
        page = pdf.get_page(page_num)
        pil_image = page.render(
            scale=1,
            rotation=0,
            crop=(0, 0, 0, 0),
            # colour=(255, 255, 255, 255),
            # annotations=True,
            # greyscale=False,
            # optimise_mode=pdfium.OptimiseMode.NONE
        ).to_pil()
        pil_image.save(os.path.join(out_dir, 'jpg', f'{filename}_{page_num}.png'))
    
    pdf.close()

In [8]:
import sys
import os
import comtypes.client
from docx2pdf import convert
from tqdm import tqdm
# filename = 0
for filename in tqdm(range(10)):
    generate_doc(filename)
    generate_pdf(os.path.join(out_dir, 'doc', f'{filename}.docx'), filename)
    generate_jpg(os.path.join(out_dir, 'pdf', f'{filename}.pdf'), filename)
    # coords, tokens = get_word_coords(os.path.join(out_dir, 'pdf', f'{filename}.pdf'))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:47<00:00,  4.77s/it]


# Comtypes

In [2]:
%%time
import sys
import os
import comtypes.client

wdFormatPDF = 17

in_file = os.path.abspath(os.path.join(out_dir, 'doc', 'letter.docx'))
out_file = os.path.abspath(os.path.join(out_dir, 'pdf', 'comtypes.pdf'))

word = comtypes.client.CreateObject('Word.Application')
doc = word.Documents.Open(in_file)
doc.SaveAs(out_file, FileFormat=wdFormatPDF)
doc.Close()
word.Quit()

# pdf2jpg.convert_pdf2jpg(out_file, os.path.join(out_dir, 'jpg', 'comtypes.jpg'), pages="ALL")

CPU times: total: 500 ms
Wall time: 3.63 s


0

In [3]:
pdf = pdfium.PdfDocument(os.path.join(out_dir, 'pdf', 'comtypes.pdf'))

n_pages = len(pdf)

for page_num in range(n_pages):
    page = pdf.get_page(page_num)
    pil_image = page.render(
        scale=1,
        rotation=0,
        crop=(0, 0, 0, 0),
        # colour=(255, 255, 255, 255),
        # annotations=True,
        # greyscale=False,
        # optimise_mode=pdfium.OptimiseMode.NONE
    ).to_pil()
    pil_image.save(os.path.join(out_dir, 'jpg', f'comtypes_{page_num}.png'))

pdf.close()

True

# Pywin32

In [4]:
%%time
import sys
import os
import win32com.client

wdFormatPDF = 17

in_file = os.path.abspath(os.path.join(out_dir, 'doc', 'letter.docx'))
out_file = os.path.abspath(os.path.join(out_dir, 'pdf', 'win32.pdf'))

word = win32com.client.Dispatch('Word.Application')
doc = word.Documents.Open(in_file)
doc.SaveAs(out_file, FileFormat=wdFormatPDF)
doc.Close()
word.Quit()

# pdf2jpg.convert_pdf2jpg(out_file, os.path.join(out_dir, 'jpg', 'win32.jpg'), pages="ALL")

CPU times: total: 31.2 ms
Wall time: 2.82 s


# Aspose.words

оставляет вотермарку

In [5]:
%%time
import aspose.words as aw

# Load word document
doc = aw.Document(os.path.join(out_dir, 'doc', 'letter.docx'))

# Save as PDF
doc.save(os.path.join(out_dir, 'pdf', 'aspose.pdf'))

# pdf2jpg.convert_pdf2jpg(os.path.join(out_dir, 'pdf', 'aspose.pdf'), os.path.join(out_dir, 'jpg', 'aspose.jpg'), pages="ALL")

CPU times: total: 2.14 s
Wall time: 3.51 s


# docx2pdf

In [6]:
%%time
from docx2pdf import convert

convert(os.path.join(out_dir, 'doc', 'letter.docx'), os.path.join(out_dir, 'pdf', 'docx2pdf.pdf'))

C:\Users\mkoltunov\Projects\letters_dataset_generator\letters_gen_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.03s/it]

CPU times: total: 109 ms
Wall time: 5.05 s


In [7]:
pdf = pdfium.PdfDocument(os.path.join(out_dir, 'pdf', 'docx2pdf.pdf'))

n_pages = len(pdf)

for page_num in range(n_pages):
    page = pdf.get_page(page_num)
    pil_image = page.render(
        scale=1,
        rotation=0,
        crop=(0, 0, 0, 0),
    ).to_pil()
    pil_image.save(os.path.join(out_dir, 'jpg', f'docx2pdf_{page_num}.png'))

pdf.close()

True

# pdf2image

## Need to download a poppler https://github.com/oschwartz10612/poppler-windows/releases/

In [57]:
# need a poppler
from pdf2image import convert_from_path

images = convert_from_path(os.path.join(out_dir, 'pdf', '99.pdf'), poppler_path=r'poppler-23.11.0\Library\bin') # set your poppler path

for i in range(len(images)):
    images[i].save(os.path.join(out_dir, 'jpg', f'docx2pdf_{i}.jpg'), 'JPEG')


In [205]:
os.path.basename(os.path.join(out_dir, 'pdf', 'letter.pdf')).split('.')[0]

'letter'